# Yolov4 Inference



# Cloning the NHD Colab Repos

The following cell determines whether you are running in colab and if the git repos should be cloned. 
Just copy and past this cell as is


In [1]:
%%bash
if [  -d "/content/nhd-colab" ]
then
   echo "Environment already initialized"
elif [ -d "/content" ] && [ ! -d "/content/nhd-colab" ]
then
    cd /content || exit 1;
    echo "Installing https://github.com/nhdchicken/nhd-colab.git"
    git clone --recurse-submodules https://github.com/nhdchicken/nhd-colab.git || exit 1;
    cd nhd-colab || exit 1;
else
    echo "Not running in Colab - going to root of repos"
    cd `git rev-parse --show-toplevel` || exit 1;
fi
pwd
pip install -e utils/nhdcolab > /dev/null 2>&1 || exit 1;
nhdcolab > /dev/null 2>&1 || exit 1;
echo "Great Success!"

Not running in Colab - going to root of repos
/Users/lpbrac/gitlab/pyops/nhd/nhd-colab
Great Success!


# Component Installation

The next cell initializes the components by running the commands specified in the [install.yaml](../install.yaml)
as well as the patches required. I not patch group is specified, all patch groups are installed. To disable patching
use --no-patch


In [2]:
!nhdcolab init yolov4-tf

NHD_COLAB_REPOS_ROOT=/Users/lpbrac/gitlab/pyops/nhd/nhd-colab OK!
Google Drive not mounted
going to repos root dir /Users/lpbrac/gitlab/pyops/nhd/nhd-colab
loading install config /Users/lpbrac/gitlab/pyops/nhd/nhd-colab/install.yml
installing components
yolov4-tf already installed - skipping


# Loading the Environment

This piece of code loads the NHD environment into your notebook. From that object, you can access various environment variables. 

If running in Colab, and if _gdrive_mount_ is True, the google drive will be mounted. On your workstation, 
it will look at _NHD_ENV.NHD_COLAB_REPOS_ROOT / 'drive' / 'nhddrive'_. 

You can however override this by setting the _NHD_COLAB_DRIVE_ environment variable to the data location. 

For instance, say your Box storage is under ~/Box/NHD/nhddrive, you can set _NHD_COLAB_DRIVE_ to that location.


In [3]:
from nhdcolab.environment import NHDEnvironment
NHD_ENV = NHDEnvironment(gdrive_mount=True)


NHD_COLAB_REPOS_ROOT=/Users/lpbrac/gitlab/pyops/nhd/nhd-colab OK!
Google drive not mounted since not running in Colab - using /Users/lpbrac/gitlab/pyops/nhd/nhd-colab/drive/nhddrive


# YoloDetector

In [5]:
import os
import sys
import click
import tensorflow as tf
ROOT_DIR = os.path.join(NHD_ENV.NHD_COLAB_REPOS_ROOT, "yolov4", "tensorf")
sys.path.append(ROOT_DIR)  # To find local version of the library
from core.config import cfg
import core.utils as utils
from core.yolov4 import YOLOv4, YOLOv3, YOLOv3_tiny, decode
import numpy as np


class YoloV4Detector:

    def __init__(self, tiny=False, version=4, quant='fp16', use_tflite=False, input_size=608, weights='yolov4.weights'):
        click.secho(f"version={version} tiny={tiny} quant={quant}", fg='cyan')
        assert weights, "Weights filename required"
        self.weights = NHD_ENV.NHD_COLAB_MODEL_ROOT / 'yolov4' / weights

        assert self.weights.is_file(), f"weights {self.weights} not found"

        self.strides = None
        self.anchors = None
        self.tiny = tiny
        self.yolo_version = version
        if tiny:
            self.strides = np.array(cfg.YOLO.STRIDES_TINY)
            self.anchors = utils.get_anchors(cfg.YOLO.ANCHORS_TINY, tiny)
        else:
            self.strides = np.array(cfg.YOLO.STRIDES)
            if version == 4:
                self.anchors = utils.get_anchors(cfg.YOLO.ANCHORS, tiny)
            else:
                self.anchors = utils.get_anchors(cfg.YOLO.ANCHORS_V3, tiny)

        self.num_classes = len(utils.read_class_names(cfg.YOLO.CLASSES))
        self.xyscale = cfg.YOLO.XYSCALE
        self.use_tflite = use_tflite
        self.input_size = input_size
        self._model = None
        self._input_layer = None

    @property
    def input_layer(self):
        if self._input_layer is None:
            click.secho(f"creating input layer {self.input_size} {self.input_size} {3}", fg='green')
            self._input_layer = tf.keras.layers.Input([self.input_size, self.input_size, 3])
        return self._input_layer

    @property
    def model(self):
        if self._model is None:
            if self.tiny:
                raise NotImplementedError("Not yet there")
            else:
                if self.yolo_version == 3:
                    raise NotImplementedError("YoloV3 not there yet")
                elif self.yolo_version == 4:
                    feature_maps = YOLOv4(self.input_layer, self.num_classes)
                    bbox_tensors = []
                    for i, fm in enumerate(feature_maps):
                        bbox_tensor = decode(fm, self.num_classes, i)
                        bbox_tensors.append(bbox_tensor)
                    self._model = tf.keras.Model(self.input_layer, bbox_tensors)
                    if self.weights.suffix == '.weights':
                        utils.load_weights(self._model, str(self.weights))
                    else:
                        self._model.load_weights(str(self.weights)).expect_partial()
                else:
                    raise ValueError(f"Invalid yolo version {self.yolo_version}")
            self._model.summary()
        return self._model

    def search(self, frame):
        image_data = utils.image_preprocess(np.copy(frame.original), [self.input_size, self.input_size])
        image_data = image_data[np.newaxis, ...].astype(np.float32)
        frame.model_input_size = self.input_size
        pred_bbox = self.model.predict(image_data)
        if self.yolo_version == 4:
            pred_bbox = utils.postprocess_bbbox(pred_bbox, self.anchors, self.strides, self.xyscale)
        else:
            pred_bbox = utils.postprocess_bbbox(pred_bbox, self.anchors, self.strides)
        bboxes = utils.postprocess_boxes(pred_bbox, frame.original_size, self.input_size, 0.25)
        frame.bboxes = utils.nms(bboxes, 0.213, method='nms')
        utils.draw_bbox(frame.original, frame.bboxes)
        return frame